# STAMP Demo — Tile Embedding Extraction from WSIs

## 1. Environment Setup

In [ ]:
import subprocess
import sys

# Helper function to run commands and capture errors
def run_command(cmd):
    try:
        result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error in command '{cmd}': {e.stderr}")
        raise

print("Setting up environment...")

# 1. Verify GPU
print("Checking GPU...")
try:
    run_command("nvidia-smi")
except Exception as e:
    print("GPU check failed. Ensure GPU is enabled (Runtime → Change runtime type → GPU).")

# 2. Install dependencies with fallback
print("Installing pandas and numpy...")
run_command("pip install -q pandas==2.2.2 numpy==1.26.4 --force-reinstall --no-warn-conflicts")

print("Updating apt and installing system packages...")
run_command("apt-get update -qq")
run_command("apt-get install -y libgl1-mesa-glx libglib2.0-0")

print("Installing Python packages...")
try:
    run_command("pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu124")
    run_command("pip install -q git+https://github.com/KatherLab/STAMP.git h5py matplotlib pillow openslide-python scikit-image --retries 3")
except Exception as e:
    print("STAMP install failed. Retrying with verbose output...")
    run_command("pip install -v -q git+https://github.com/KatherLab/STAMP.git h5py matplotlib pillow openslide-python scikit-image --retries 3")

# 3. Mount Drive
print("Mounting Google Drive...")
from google.colab import drive
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Drive mount failed: {e}")
    print("Please authenticate when prompted and paste the code.")

# 4. Setup output directory and verify GPU
import os, matplotlib.pyplot as plt, h5py, numpy as np, torch
os.makedirs("/content/drive/MyDrive/STAMP_demo_features", exist_ok=True)
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print("Environment setup complete!")

Setting up environment...
Checking GPU...
Wed Oct 15 04:09:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----

In [ ]:
# Using WSI from /content/drive/MyDrive/WSI_Shared_Folder/output_imgs/high_risk/TCGA-2A-A8VT
!ls -lh "/content/drive/MyDrive/WSI_Shared_Folder/output_imgs/high_risk/TCGA-2A-A8VT/TCGA-2A-A8VT-01Z-00-DX1.F2CD9AAB-7B40-46A4-832B-1279A8A77737.svs"

-rw------- 1 root root 1.5G Jun  8  2024 /content/drive/MyDrive/WSI_Shared_Folder/output_imgs/high_risk/TCGA-2A-A8VT/TCGA-2A-A8VT-01Z-00-DX1.F2CD9AAB-7B40-46A4-832B-1279A8A77737.svs


## 2. Create Config File

In [ ]:
## Create Config File
%%bash
cat > config.yaml <<EOF
preprocessing:
  output_dir: "/content/drive/MyDrive/STAMP_demo_features"
  wsi_dir: "/content/drive/MyDrive/WSI_Shared_Folder/output_imgs/high_risk/TCGA-2A-A8VT"

  # Other possible values are "mahmood-uni" and "mahmood-conch"
  extractor: "ctranspath"

  # Having a cache dir will speed up extracting features multiple times,
  # e.g. with different feature extractors.
  # Optional.
  # cache_dir: "/absolute/path/to/stamp-test-experiment/../cache"
  # If you do not want to use a cache,
  # change the cache dir to the following:
  cache_dir: "/content/drive/MyDrive/STAMP_demo_features/cache"

  # Device to run feature extraction on.
  # Set this to "cpu" if you do not have a CUDA-capable GPU.
  device: "cuda"

  # How many workers to use for tile extraction.  Should be less or equal to
  # the number of cores of your system.
  max_workers: 8

EOF

cat config.yaml

preprocessing:
  output_dir: "/content/drive/MyDrive/STAMP_demo_features"
  wsi_dir: "/content/drive/MyDrive/WSI_Shared_Folder/output_imgs/high_risk/TCGA-2A-A8VT"

  # Other possible values are "mahmood-uni" and "mahmood-conch"
  extractor: "ctranspath"

  # Having a cache dir will speed up extracting features multiple times,
  # e.g. with different feature extractors.
  # Optional.
  # cache_dir: "/absolute/path/to/stamp-test-experiment/../cache"
  # If you do not want to use a cache,
  # change the cache dir to the following:
  cache_dir: null

  # Device to run feature extraction on.
  # Set this to "cpu" if you do not have a CUDA-capable GPU.
  device: "cuda"

  # How many workers to use for tile extraction.  Should be less or equal to
  # the number of cores of your system.
  max_workers: 8



In [ ]:
!ls config.yaml

config.yaml


## 3. Preprocess (Extract Tiles)

In [ ]:
!stamp --config config.yaml preprocess

2025-10-15 04:32:09,707	INFO	using the following configuration:
brightness_cutoff: 240
cache_dir: null
cache_tiles_ext: jpg
canny_cutoff: 0.02
default_slide_mpp: null
device: cuda
extractor: ctranspath
generate_hash: true
max_workers: 8
output_dir: /content/drive/MyDrive/STAMP_demo_features
tile_size_px: 224
tile_size_um: 256.0
wsi_dir: /content/drive/MyDrive/WSI_Shared_Folder/output_imgs/high_risk/TCGA-2A-A8VT
wsi_list: null

Downloading...
From (original): https://drive.google.com/u/0/uc?id=1DoDx_70_TLj98gTf6YTXnu4tFhsFocDX&export=download
From (redirected): https://drive.google.com/uc?id=1DoDx_70_TLj98gTf6YTXnu4tFhsFocDX&export=download&confirm=t&uuid=8f73572d-d792-4c5c-b9d9-be0527dae21a
To: /root/.cache/stamp/ctranspath.pth
100% 111M/111M [00:02<00:00, 50.5MB/s]
2025-10-15 04:32:17,556	INFO	Using extractor ctranspath
TCGA-2A-A8VT-01Z-00-DX1.F2CD9AAB-7B40-46A4-832B-1279A8A77737.svs:   0% 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [06:46, 406.15s/it]
2it [07:14, 184.12s/it]
3it [08: